In [26]:
from pyspark.sql import SparkSession
spark  = SparkSession.builder.appName('mllib').getOrCreate()

In [27]:
spark

In [28]:
df_pyspark= spark.read.csv("test1.csv",header=True,inferSchema=True)

In [29]:
df_pyspark.show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  vicky| 31|        10| 30000|
|vignesh| 30|         8| 25000|
|    ram| 29|         4| 20000|
|   somu| 24|         3| 20000|
|  vijay| 21|         1| 15000|
|  surya| 23|         2| 18000|
+-------+---+----------+------+



In [30]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [31]:
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

In [32]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")


In [33]:
output=featureassembler.transform(df_pyspark)

In [34]:
output.show()

+-------+---+----------+------+--------------------+
|   Name|age|Experience|Salary|Independent Features|
+-------+---+----------+------+--------------------+
|  vicky| 31|        10| 30000|         [31.0,10.0]|
|vignesh| 30|         8| 25000|          [30.0,8.0]|
|    ram| 29|         4| 20000|          [29.0,4.0]|
|   somu| 24|         3| 20000|          [24.0,3.0]|
|  vijay| 21|         1| 15000|          [21.0,1.0]|
|  surya| 23|         2| 18000|          [23.0,2.0]|
+-------+---+----------+------+--------------------+



In [35]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [36]:
finalized_data=output.select("Independent Features","Salary")


In [37]:
finalized_data.show()


+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [44]:
# train a model

from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

23/11/30 14:14:44 WARN Instrumentation: [0396d4f1] regParam is zero, which might cause numerical instability and overfitting.


In [45]:
train_data.show()
test_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [21.0,1.0]| 15000|
|          [29.0,4.0]| 20000|
|         [31.0,10.0]| 30000|
+--------------------+------+

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [23.0,2.0]| 18000|
|          [24.0,3.0]| 20000|
|          [30.0,8.0]| 25000|
+--------------------+------+



In [46]:

### Coefficients
regressor.coefficients

DenseVector([-0.0, 1666.6667])

In [47]:
### Intercepts
regressor.intercept

13333.333333333807

In [48]:
# Prediction
pred_results=regressor.evaluate(test_data)

In [49]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16666.666666666693|
|          [24.0,3.0]| 20000| 18333.33333333336|
|          [30.0,8.0]| 25000|26666.666666666664|
+--------------------+------+------------------+



/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [50]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError


(1555.5555555555366, 2444444.4444443868)